# FF Convergence

In [5]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import tensorflow as tf
import random
import pickle as pkl

from qiskit.quantum_info import Operator, Statevector, random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from quantum_tools import circuit_to_matrix, partial_trace
from kraus_channels import isomery_to_kraus, KrausMap
from quantum_channel import ConcatChannel, DilutedChannel, IdentityChannel, ChoiMapStatic
from utils import numberToBase, kron
from spectrum import complex_spacing_ratio, channel_spectrum, Spectrum, mean_spacing, coat_spectrum
from analysis import spectrum_to_angular, spectrum_to_radial, hist_ensamble
from qiskit.circuit.library import iSwapGate, XGate, YGate, HGate, CXGate, RGate, RZGate
from quantum_circuits import integrable_circuit, nonintegrable_circuit, haar_random
from synthetic_data import generate_map_data, generate_spam_data, generate_spam_benchmark
from optimization import ModelQuantumMap, ModelSPAM, Logger, model_saver
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from spam import SPAM, InitialState, POVMwQR as POVM, CorruptionMatrix, IdealSPAM

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=1)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


## Circuits

In [2]:
def sample_traceout_csr(n, n_anc, reps, circuit_type, L):
    channel_list = []
    spectra_list = []
    csr_list = []


    for _ in tqdm(range(reps)):
        circuit = circuit_type(n+n_anc, L)
        U = Operator(circuit).data

        kraus_map = isomery_to_kraus(U, rank = 2**n_anc)
        channel_list.append(kraus_map)
        
        spectrum = channel_spectrum(kraus_map)
        spectra_list.append(spectrum)
        csr = complex_spacing_ratio(spectrum, log=False)
        csr_list.append(csr)

    return  channel_list, spectra_list, csr_list

In [3]:
def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets,
              channel, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue(), channel_fidelity_loss], sample_freq=100),
                )
    model.train(inputs = inputs, 
                targets = targets,
                inputs_val = [inputs, None],
                targets_val = [targets, [channel]],
                N=500,
                num_iter = num_iter,
                verbose = verbose,)
    
    return model

## Free Fermion

### Generate Benchmarks

In [4]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/FF_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [ ]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in channel_FF_list:
    
    spam_target = IdealSPAM(d)

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 12000)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_target,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_FF_noisy_noSpam.model")


[0.9948856964484133, -0.9850668282176864]
[0.9962156630428204, -0.9854777772857295]
[0.9956537781021114, -0.9851699920049471]
[0.9950757985389802, -0.9843501901493977]
[0.9955827954603652, -0.9853946584632594]
[0.9957491013073639, -0.9849208537659377]
[0.9961129536846139, -0.9864386776373247]
[0.9954508967746061, -0.9847494501981616]
[0.9955512047115922, -0.9844991360141891]
[0.9959533321477326, -0.985193914176386]


## Chaotic Haar Random

### Generate Benchmarks

In [7]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/chaotic_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [8]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in tqdm(channel_FF_list):
    
    spam_target = generate_spam_benchmark(n=4, c1=0.95, c2=0.95)
    inputs_spam, targets_spam = generate_spam_data(spam_target, shots=4096)
    
    spam_model = fit_spam(inputs_spam,
                        targets_spam,
                        num_iter = 3000,
                        verbose = False)
        

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 4096)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_model,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_chaotic_noisy.model")


  0%|          | 0/10 [00:00<?, ?it/s]

[0.9904482282721974, -0.9823848497374638]
[0.9903797801123899, -0.9829675553329221]
[0.9904841635992646, -0.9820980688231122]
[0.9904696364347323, -0.9820669953989726]
[0.990575640099721, -0.9813795070842284]
[0.9904794955280019, -0.981610035841893]
[0.9904959132417168, -0.981645976993597]
[0.9905007192722082, -0.9823577484652025]
[0.9904565560967117, -0.9821516646215387]
[0.9902642862801311, -0.9812095452091081]
